# PART 1 
## Installing and Importing programs to read and parse html document

In [1]:
import requests

In [2]:
!pip install BeautifulSoup4
!pip install lxml

#### Passing website which contains a list of neighboorhoods, boroughs in Canada and their postal codes 

In [80]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#soup.find("table",{"class":"wikitable"})


<table class="wikitable sortable">
<tbody><tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park / Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor / Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park / Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern / Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>Ea

#### Using tag 'td' to find and split values from the website. 
#### Assign values to the empty lists (i,e., code_list)
#### Grouping information into a table using pandas

In [5]:
import pandas as pd
codes_list=[]
borough_list=[]
neighborhood_list=[]
i=1
for tag in soup.table.find_all('td'):
    if i == 1:
        tag = tag.text
        tag = tag.replace('\n', '')
        codes_list.append(tag)
        
    if i == 2:
        tag = tag.text
        tag = tag.replace('\n', '')
        borough_list.append(tag)
    if i == 3: 
        tag = tag.text
        tag = tag.replace('\n', '')
        neighborhood_list.append(tag)
    i = i+1
    if i==4:
        i=1
#combining lists into dataframe
df = pd.DataFrame()
df['PostalCode']=codes_list
df['Borough']=borough_list 
df['Neighborhood']=neighborhood_list
#replacing / with ,
df=df.replace({'/':','},regex=True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo..."


#### Dropping not assigned Borough and reseting index

In [6]:
df = df[df['Borough'] != 'Not assigned'].reset_index(drop=True)
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
df.shape

(103, 3)

#### The table above contains 103 neighborhoods

### PART 2

##### installing geocoder to get coordinates of the 103 neighborhoods

In [8]:
!conda install -c conda-forge geocoder --yes

Solving environment: ...working... done

# All requested packages already installed.



In [9]:
import geocoder

#### Obtaning latitute and longitude coordinates of neighborhoods using the PostalCode column

In [10]:
latitude=[]
longitude=[]
# select each row of df 
for i in range(len(df)):
#initialize lat_lng_coords to none
    lat_lng_coords = None    
#loop until you get the coordinates
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{},Toronto, Ontario'.format(df['PostalCode'][i]))
        lat_lng_coords=g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
#adding latitude and longitude columns to table
df['Latitude']=latitude
df['Longitude']=longitude


In [11]:
print(df.shape)
df.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.661790,-79.389390


### PART 3

##### Selecting Boroughs that contain the word 'York' in their names

In [12]:
df_York=df[df['Borough'].str.contains('York')].reset_index(drop=True)
print(df_York.shape)
df_York.head(10)


(34, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211
3,M3B,North York,Don Mills,43.748900,-79.357220
4,M4B,East York,"Parkview Hill , Woodbine Gardens",43.707193,-79.311529
5,M6B,North York,Glencairn,43.707279,-79.447500
6,M3C,North York,Don Mills,43.722143,-79.352023
7,M4C,East York,Woodbine Heights,43.689740,-79.308507
8,M6C,York,Humewood-Cedarvale,43.691730,-79.430013
9,M6E,York,Caledonia-Fairbanks,43.689118,-79.450650


##### Installing folium needed to visualize neighborhoods in a map

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


In [114]:
website_url2 = "https://en.wikipedia.org/wiki/Toronto_subway"
table = pd.read_html(website_url2)[2]
table

,Line,Stations,Length[1][10],Technology
0,Yonge–University,38,38.8 km (24.1 mi),Heavy rail
1,Bloor–Danforth,31,26.2 km (16.3 mi),Heavy rail
2,Scarborough,6,6.4 km (4.0 mi),Light metro
3,Sheppard,5,5.5 km (3.4 mi),Heavy rail
4,Under construction,Under construction,Under construction,Under construction
5,Eglinton,25,19 km (12 mi),Light rail
6,Finch West,18,11 km (6.8 mi),Light rail


In [110]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_volcanoes_by_elevation#From_its_base_on_the_ocean_floor").text
soup3 = BeautifulSoup(page, "lxml")
#tab=soup3.find("table",{"class":"wikitable"})
#print(tab)
print(soup3.title.text)
line=[]
off_peak=[]
rush_hour=[]
e=[]
#f=[]
#g=[]
#h=[]

i=1
for tag in soup3.table.find_all('td'):
    if i == 1:
        tag = tag.text
        tag = tag.replace('\n', '')
        line.append(tag)
        
    if i == 2:
        tag = tag.text
        tag = tag.replace('\n', '')
        off_peak.append(tag)
          
    if i == 3: 
        tag = tag.text
        tag = tag.replace('\n', '')
        rush_hour.append(tag)
    if i == 4: 
        tag = tag.text
        tag = tag.replace('\n', '')
        e.append(tag)
    #if i == 5: 
        #tag = tag.text
        #tag = tag.replace('\n', '')
        #f.append(tag)
    #if i == 6: 
        #tag = tag.text
        #tag = tag.replace('\n', '')
        #g.append(tag)
    #if i == 7: 
        #tag = tag.text
        #tag = tag.replace('\n', '')
        #h.append(tag)
          
    i = i+1
    if i==5:
        i=1
#combining lists into dataframe
df2 = pd.DataFrame()
df2['line_color']=line
df2['offpeak']=off_peak
df2['rushhour']=rush_hour
df2['re']=e
#df2['rf']=f
#df2['rg']=g
#df2['rh']=h
#replacing / with ,
#df2=df2.replace({'/':','},regex=True)
df2

List of volcanoes by elevation - Wikipedia


,line_color,offpeak,rushhour,re
0,Mauna Kea,"10,203","33,474","33474 – Hawaii, U.S.A"


##### Map of all neighborhoods whose Borough contains the word 'York' only

In [15]:

# create map of Manhattan using latitude and longitude values
map_df_York=folium.Map(location=[43.7001114, -79.4162979], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_York['Latitude'], df_York['Longitude'], df_York['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_df_York)  
    
map_df_York

##### Getting venues 500 meters from the neighborhoods

In [20]:
# @hidden_cell
CLIENT_ID =  'AANQS4UMGB0ZZS43JRAIHXEU0J3BYFFC2RAGGCU0K1XUVYFA' # your Foursquare ID
CLIENT_SECRET = 'JJNSOBEPGUMRJOHRUPS2V4CIOKS0UXPRKSDMWBJSCUXQ1ZDA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


##### Defining function getNerbyVenues and using Foursquare API to gather up 100 venues that are 500 meters from each neighboorhood

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Calling getNerbyVenues and counting the number of unique venue categories retrieved

In [22]:
york_venues = getNearbyVenues(names=df_York['Neighborhood'],
                                   latitudes=df_York['Latitude'],
                                   longitudes=df_York['Longitude']
                                  )
print('\n There are {} uniques categories.'.format(len(york_venues['Venue Category'].unique())))

Parkwoods
Victoria Village
Lawrence Manor , Lawrence Heights
Don Mills
Parkview Hill , Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Bayview Village
Downsview
York Mills , Silver Hills
Downsview
North Park , Maple Leaf Park , Upwood Park
Humber Summit
Willowdale , Newtonbrook
Downsview
Bedford Park , Lawrence Manor East
Del Ray , Mount Dennis , Keelsdale and Silverthorn
Humberlea , Emery
Willowdale
Downsview
Runnymede , The Junction North
Weston
York Mills West
Willowdale

 There are 130 uniques categories.


##### 2856 venues were retrieved. There are 130 unique venue categories

In [29]:
print('Total number of venues retrieved: ',york_venues.size)
york_venues.head()

Total number of venues retrieved:  2856


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.752935,-79.335641,MacLeod Exteriors Inc.,43.755014,-79.338688,Construction & Landscaping
3,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant


##### Rearrenging table to show "Venue Catergories" as headers

In [34]:
york_onehot = pd.get_dummies(york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
york_onehot['Neighborhood'] = york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [york_onehot.columns[-1]] + list(york_onehot.columns[:-1])
york_onehot = york_onehot[fixed_columns]

york_onehot.head(7)


,Neighborhood,Airport,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Baby Store,Bagel Shop,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each venue category

In [39]:
york_grouped = york_onehot.groupby('Neighborhood').mean().reset_index()
york_grouped.head()

,Neighborhood,Airport,American Restaurant,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Baby Store,Bagel Shop,Bakery,Bank,...,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Video Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,"Bathurst Manor , Wilson Heights , Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.105263,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.0,0.000000,0.0
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.0,0.000000,0.0
2,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.045455,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
3,Caledonia-Fairbanks,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.111111,0.0
4,"Del Ray , Mount Dennis , Keelsdale and Silvert...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


##### Defining function return_most_common_venues to obtain the top venues for each neighborhood

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Calling function and rearrenging table to show the top 10 venues 

In [45]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = york_grouped['Neighborhood']

for ind in np.arange(york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Pharmacy,Pizza Place,Deli / Bodega,Diner,Restaurant,Bridal Shop,Sandwich Place
1,Bayview Village,Flower Shop,Trail,Construction & Landscaping,Gym,Grocery Store,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop
2,"Bedford Park , Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Sushi Restaurant,Cosmetics Shop,Restaurant,Pub,Juice Bar,Butcher,Sports Club
3,Caledonia-Fairbanks,Park,Gym,Bakery,Mexican Restaurant,Spa,Beer Store,Sporting Goods Shop,Women's Store,Gas Station,Gastropub
4,"Del Ray , Mount Dennis , Keelsdale and Silvert...",Construction & Landscaping,Coffee Shop,Distribution Center,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office,Yoga Studio


#### Clustering Neighborhoods based on their top 10 venue categories using 3 clusters

In [46]:
from sklearn.cluster import KMeans 
# set number of clusters
kclusters = 3

york_grouped_clustering = york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 2, 0, 0, 0, 0, 1])

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

york_merged = df_York

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
york_merged = york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

york_merged.head(10) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,1,Construction & Landscaping,Food & Drink Shop,Park,Distribution Center,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office,Yoga Studio
1,M4A,North York,Victoria Village,43.728102,-79.311890,0,Pizza Place,Coffee Shop,Portuguese Restaurant,Park,Intersection,French Restaurant,Discount Store,Electronics Store,Dog Run,Doctor's Office
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723265,-79.451211,0,Clothing Store,American Restaurant,Furniture / Home Store,Restaurant,Toy / Game Store,Food Court,Men's Store,Bookstore,Cosmetics Shop,Pharmacy
3,M3B,North York,Don Mills,43.748900,-79.357220,0,Athletics & Sports,Gym,Bank,Park,Coffee Shop,Other Great Outdoors,Restaurant,Burger Joint,Spa,Construction & Landscaping
6,M3C,North York,Don Mills,43.722143,-79.352023,0,Athletics & Sports,Gym,Bank,Park,Coffee Shop,Other Great Outdoors,Restaurant,Burger Joint,Spa,Construction & Landscaping
4,M4B,East York,"Parkview Hill , Woodbine Gardens",43.707193,-79.311529,0,Pizza Place,Fast Food Restaurant,Café,Pet Store,Athletics & Sports,Gym / Fitness Center,Gastropub,Rock Climbing Spot,Intersection,Bank
5,M6B,North York,Glencairn,43.707279,-79.447500,0,Pizza Place,Gas Station,Pub,Rental Car Location,Mediterranean Restaurant,Japanese Restaurant,Fast Food Restaurant,Grocery Store,Furniture / Home Store,Gift Shop
7,M4C,East York,Woodbine Heights,43.689740,-79.308507,0,Pharmacy,Gas Station,Breakfast Spot,Doctor's Office,Bar,Sushi Restaurant,Grocery Store,Coffee Shop,Arts & Crafts Store,Department Store
8,M6C,York,Humewood-Cedarvale,43.691730,-79.430013,0,Hockey Arena,Business Service,Grocery Store,Trail,Park,Field,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office
9,M6E,York,Caledonia-Fairbanks,43.689118,-79.450650,0,Park,Gym,Bakery,Mexican Restaurant,Spa,Beer Store,Sporting Goods Shop,Women's Store,Gas Station,Gastropub


####  Visualize the resulting clusters

In [48]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.7001114, -79.4162979], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(york_merged['Latitude'], york_merged['Longitude'], york_merged['Neighborhood'], york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 0 red color. Neigborhoods like Parks

In [49]:
york_merged.loc[york_merged['Cluster Labels'] == 0, york_merged.columns[[2] + list(range(5, york_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,0,Pizza Place,Coffee Shop,Portuguese Restaurant,Park,Intersection,French Restaurant,Discount Store,Electronics Store,Dog Run,Doctor's Office
2,"Lawrence Manor , Lawrence Heights",0,Clothing Store,American Restaurant,Furniture / Home Store,Restaurant,Toy / Game Store,Food Court,Men's Store,Bookstore,Cosmetics Shop,Pharmacy
3,Don Mills,0,Athletics & Sports,Gym,Bank,Park,Coffee Shop,Other Great Outdoors,Restaurant,Burger Joint,Spa,Construction & Landscaping
6,Don Mills,0,Athletics & Sports,Gym,Bank,Park,Coffee Shop,Other Great Outdoors,Restaurant,Burger Joint,Spa,Construction & Landscaping
4,"Parkview Hill , Woodbine Gardens",0,Pizza Place,Fast Food Restaurant,Café,Pet Store,Athletics & Sports,Gym / Fitness Center,Gastropub,Rock Climbing Spot,Intersection,Bank
5,Glencairn,0,Pizza Place,Gas Station,Pub,Rental Car Location,Mediterranean Restaurant,Japanese Restaurant,Fast Food Restaurant,Grocery Store,Furniture / Home Store,Gift Shop
7,Woodbine Heights,0,Pharmacy,Gas Station,Breakfast Spot,Doctor's Office,Bar,Sushi Restaurant,Grocery Store,Coffee Shop,Arts & Crafts Store,Department Store
8,Humewood-Cedarvale,0,Hockey Arena,Business Service,Grocery Store,Trail,Park,Field,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office
9,Caledonia-Fairbanks,0,Park,Gym,Bakery,Mexican Restaurant,Spa,Beer Store,Sporting Goods Shop,Women's Store,Gas Station,Gastropub
10,Leaside,0,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Sports Bar,Burger Joint,Bank,Pet Store,Liquor Store,Juice Bar,Shopping Mall


#### Cluster 1 purple color. Neigborhoods like Coffe Shops

In [50]:
york_merged.loc[york_merged['Cluster Labels'] == 1, york_merged.columns[[2] + list(range(5, york_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,1,Construction & Landscaping,Food & Drink Shop,Park,Distribution Center,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office,Yoga Studio
11,Hillcrest Village,1,Park,Residential Building (Apartment / Condo),Dog Run,Yoga Studio,Field,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner
21,"North Park , Maple Leaf Park , Upwood Park",1,Bakery,Park,Basketball Court,Distribution Center,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office,Yoga Studio
27,"Humberlea , Emery",1,Playground,Park,Field,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
32,York Mills West,1,Park,Convenience Store,Bank,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office,Distribution Center,Discount Store


#### Cluster 2 teal color. Neigborhoods like Construction & Landscaping

In [51]:
york_merged.loc[york_merged['Cluster Labels'] == 2, york_merged.columns[[2] + list(range(5, york_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,"Del Ray , Mount Dennis , Keelsdale and Silvert...",2,Construction & Landscaping,Coffee Shop,Distribution Center,Field,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Dog Run,Doctor's Office,Yoga Studio


In [1]:
from pandas import numpy as np

ImportError: cannot import name 'numpy' from 'pandas' (C:\Users\hp\anaconda3\lib\site-packages\pandas\__init__.py)

In [2]:
import numpy as np

In [4]:
np.arange(10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])